In [170]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
import glob
import re
import os
import numpy as np
import sys

Stopwords = set(stopwords.words('english'))

In [171]:
def finding_all_unique_words_and_freq(words):
    words_unique = []
    word_freq = {}
    for word in words:
        if word not in words_unique:
            words_unique.append(word)
    for word in words_unique:
        word_freq[word] = words.count(word)
    return word_freq


def finding_freq_of_word_in_doc(word, words):
    freq = words.count(word)


def remove_special_characters(text):
    regex = re.compile('[^a-zA-Z0-9\s]')
    text_returned = re.sub(regex, '', text)
    return text_returned

In [172]:
all_words = []
dict_global = {}
file_folder = 'data/*.txt'
idx = 1
files_with_index = {}
for file in glob.glob(file_folder):
    print(file)
    fname = file
    file = open(file, "r")
    text = file.read()
    text = remove_special_characters(text)
    text = re.sub(re.compile('\d'), '', text)
    sentences = sent_tokenize(text)
    words = word_tokenize(text)
    words = [word for word in words if len(words) > 1]
    words = [word.lower() for word in words]
    words = [word for word in words if word not in Stopwords]
    dict_global.update(finding_all_unique_words_and_freq(words))
    files_with_index[idx] = os.path.basename(fname)
    idx = idx + 1

unique_words_all = set(dict_global.keys())

data/doc4.txt
data/doc1.txt
data/doc3.txt
data/doc2.txt


In [173]:
class Node:
    def __init__(self, docId, freq=None):
        self.freq = freq
        self.doc = docId
        self.nextval = None


class SlinkedList:
    def __init__(self, head=None):
        self.head = head

In [174]:
linked_list_data = {}
for word in unique_words_all:
    linked_list_data[word] = SlinkedList()
    linked_list_data[word].head = Node(1, Node)
word_freq_in_doc = {}
idx = 1
for file in glob.glob(file_folder):
    file = open(file, "r")
    text = file.read()
    text = remove_special_characters(text)
    text = re.sub(re.compile('\d'), '', text)
    sentences = sent_tokenize(text)
    words = word_tokenize(text)
    words = [word for word in words if len(words) > 1]
    words = [word.lower() for word in words]
    words = [word for word in words if word not in Stopwords]
    word_freq_in_doc = finding_all_unique_words_and_freq(words)
    for word in word_freq_in_doc.keys():
        linked_list = linked_list_data[word].head
        while linked_list.nextval is not None:
            linked_list = linked_list.nextval
        linked_list.nextval = Node(idx, word_freq_in_doc[word])
    idx = idx + 1

In [175]:
query = input('Enter your query:')
print(query)
query = query.split()  # Split the query into words
connecting_words = []
different_words = []
bitwise_op = []

# Process the query to separate connecting words (AND, OR, NOT) and different words
for word in query:
    if word.lower() in ["and", "or", "not"]:
        connecting_words.append(word.lower())
    else:
        different_words.append(word.lower())

print(connecting_words)
total_files = len(files_with_index)
zeroes_and_ones_of_all_words = []

# Process the different words in the query
for word in different_words:
    if word in unique_words_all:
        zeroes_and_ones = [0] * total_files
        linkedlist = linked_list_data[word].head
        while linkedlist.nextval is not None:
            zeroes_and_ones[linkedlist.nextval.doc - 1] = 1
            linkedlist = linkedlist.nextval
        zeroes_and_ones_of_all_words.append(zeroes_and_ones)
    else:
        print(word, " not found")
        sys.exit()

print(zeroes_and_ones_of_all_words)

# Perform AND, OR, NOT operations based on connecting words
for word in connecting_words:
    if word == "and":
        if len(zeroes_and_ones_of_all_words) >= 2:
            bitwise_op = [w1 & w2 for (w1, w2) in zip(
                zeroes_and_ones_of_all_words[0], zeroes_and_ones_of_all_words[1])]
            zeroes_and_ones_of_all_words.pop(0)
            zeroes_and_ones_of_all_words.pop(0)
            zeroes_and_ones_of_all_words.insert(0, bitwise_op)
        else:
            print("Not enough operands for 'AND' operation.")
            sys.exit()
    elif word == "or":
        if len(zeroes_and_ones_of_all_words) >= 2:
            bitwise_op = [w1 | w2 for (w1, w2) in zip(
                zeroes_and_ones_of_all_words[0], zeroes_and_ones_of_all_words[1])]
            zeroes_and_ones_of_all_words.pop(0)
            zeroes_and_ones_of_all_words.pop(0)
            zeroes_and_ones_of_all_words.insert(0, bitwise_op)
        else:
            print("Not enough operands for 'OR' operation.")
            sys.exit()
    elif word == "not":
        if len(zeroes_and_ones_of_all_words) >= 1:
            bitwise_op = [int(not w) for w in zeroes_and_ones_of_all_words[0]]
            zeroes_and_ones_of_all_words.pop(0)
            zeroes_and_ones_of_all_words.insert(0, bitwise_op)
        else:
            print("Not enough operands for 'NOT' operation.")
            sys.exit()

files = []
if zeroes_and_ones_of_all_words:
    lis = zeroes_and_ones_of_all_words[0]
    cnt = 1
    for index in lis:
        if index == 1:
            files.append(files_with_index[cnt])
        cnt = cnt + 1

print(files)

Home AND July AND NOT Sales
['and', 'and', 'not']
[[1, 1, 1, 1], [1, 0, 1, 1], [1, 1, 1, 1]]
['doc1.txt']
